In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

import the data

In [ ]:
Data=pd.read_csv("/kaggle/input/voicegender/voice.csv")

In [ ]:
Data

We can see our data above.

Normalization:

the goal of normalization is to change the values of numeric columns in the dataset to use a common scale, without distorting differences in the ranges of values or losing information. Normalization is also required for some algorithms to model the data correctly.

In [ ]:
Data.label=[1 if each == "male" else 0 for each in Data.label]
y=Data.label.values
x=Data.drop(["label"], axis=1)

In [ ]:
y

In [ ]:
x=(x-np.min(x))/(np.max(x)-np.min(x)).values

In [ ]:
x

Train Test Split


We divide our data set as train and test set. Test size is 20%.

Random state: Every time the data set is split, it is split from elsewhere. 
Random_state is used to prevent this and to  divide data from the same place.


In [ ]:
from sklearn.model_selection import train_test_split


In [ ]:
#here we create x_train, y_train, x_test y_test arrays
x_train, x_test, y_train, y_test=train_test_split(x, y, test_size=0.2, random_state=42)

x_train = x_train.T
x_test = x_test.T
y_train = y_train.T
y_test = y_test.T

Initializing Parameters

In [ ]:
#We initialize weights and bias (w,b)
def initialize_weights_bias(dimension):
    w=np.full((dimension,1),0.01)
    b=0.0
    return w,b

Forward Propagation:

Forward propagation  refers to the calculation and storage of intermediate variables (including outputs) for a neural network in order from the input layer to the output layer.

In [ ]:
#calculation of z(sigmoid function)
def sigmoid(z):
    y_head=1/(1+np.exp(-z))
    return y_head


In [ ]:
y_head=sigmoid(0)
y_head

In [ ]:
#implementing forward propagation
def forward_propagation(w,b,x_train,y_train):
    z = np.dot(w.T,x_train) + b
    y_head = sigmoid(z)
    loss = -(y_train*np.log(y_head)+(1-y_train)*np.log(1-y_head))
    cost = np.sum(loss)/(x_train.shape[1]) 
    return cost


Gradient Descent

 if cost is high it means that we make wrong prediction. 
 So we need to decrease cost. 
 In order to decrease cost, we need to update weights and bias.
 So we use gradient descent.
 Gradient descent is an optimization algorithm used to find the values of parameters (coefficients) of a function (f) that minimizes a cost function (cost).

In [ ]:
def forward_backward_propagation(w,b,x_train,y_train):
    z = np.dot(w.T,x_train) + b
    y_head = sigmoid(z)
    loss = -(y_train*np.log(y_head)+(1-y_train)*np.log(1-y_head))
    cost = np.sum(loss)/(x_train.shape[1])
    #backward propagation
    derivative_weight=(np.dot(x_train,((y_head-y_train).T)))/x_train.shape[1]
    derivative_bias=np.sum(y_head-y_train)/x_train.shape[1]
    gradients={"derivative_weight" : derivative_weight,"derivative_bias" : derivative_bias}
    return cost,gradients


In [ ]:
#updating parameters
def update(w,b,x_train,y_train,learning_rate, number_of_iteration):
    cost_list= []
    
    index= []
    
    for i in range(number_of_iteration):
        cost,gradients= forward_backward_propagation(w,b,x_train,y_train)
        
        w = w - learning_rate * gradients["derivative_weight"]
        b = b - learning_rate * gradients["derivative_bias"]
        if i%100==0:
            cost_list.append(cost)
            index.append(i)
        
        print ("Cost after iteration %i: %f" %(i, cost))
    # we update(learn) parameters weights and bias
    
    parameters = {"derivative_weight": w,"derivative_bias": b}
    plt.plot(index,cost_list)
    plt.xlabel("Number of iteration")
    plt.ylabel("Cost")
    plt.show()
    return parameters, gradients, cost_list

    
    

Prediction

In [ ]:
def predict(w,b,x_test):
    z=sigmoid(np.dot(w.T,x_test)+b)
    y_prediction=np.zeros((1,x_test.shape[1]))
    
    for i in range (z.shape[1]):
        if z[0,i]<=0.5:
            y_prediction[0,i]==0
        else:
            y_prediction[0,i]==1
            
    return y_prediction
            
    

Put them all together!!!

In [ ]:
def Logistic_Regression(x_train, y_train, x_test, y_test,learning_rate, number_of_iteration):
    dimension=x_train.shape[0]
    
    w,b=initialize_weights_bias(dimension)
    parameters, gradient, cost_list=update(w, b, x_train, y_train, learning_rate,number_of_iteration)
    
    y_prediction_test= predict(parameters["derivative_weight"],parameters["derivative_bias"],x_test)
    y_prediction_train= predict(parameters["derivative_weight"],parameters["derivative_bias"],x_train)
    
    print("train accuracy: {} %".format(100 - np.mean(np.abs(y_prediction_train - y_train)) * 100))
    print("test accuracy: {} %".format(100 - np.mean(np.abs(y_prediction_test - y_test)) * 100))
    

    


In [ ]:
Logistic_Regression(x_train, y_train, x_test, y_test,learning_rate = 0.1, number_of_iteration = 3000)